<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task2/blob/main/LSTM_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Carga de las librerías y paquetes necesarios

import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
# Datos de entrenamiento
import pandas as pd

train = pd.read_csv("/content/EXIST2021_training.tsv", delimiter='\t', usecols=['language','text', 'task2'], quoting=3, error_bad_lines=False)
test = pd.read_csv("/content/EXIST2021_test_labeled.tsv", delimiter='\t', usecols=['language','text', 'task2'], quoting=3, error_bad_lines=False)

<ipython-input-3-113f6e7b2ced>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  train = pd.read_csv("/content/EXIST2021_training.tsv", delimiter='\t', usecols=['language','text', 'task2'], quoting=3, error_bad_lines=False)
<ipython-input-3-113f6e7b2ced>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test = pd.read_csv("/content/EXIST2021_test_labeled.tsv", delimiter='\t', usecols=['language','text', 'task2'], quoting=3, error_bad_lines=False)


In [4]:
# Pre-procesamiento par el idioma Inglés

import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import pandas as pd

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Función para mapear las etiquetas POS a las etiquetas de WordNet

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Función para preprocesar el texto

def preprocess_text(text):
    # Eliminar usuarios identificados por @
    text = re.sub(r'@\S+', '', text)

    # Eliminar URL's y links
    text = re.sub(r'https?:\/\/\S+', '', text)

    text = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', text)

    # Elimina referencias link o video

    text = re.sub(r'{link}', '', text)
    text = re.sub(r"\[video\]", '', text)

    # Elimina entidades HTML
    text = re.sub(r'&[a-z]+;', '', text)

    # Eliminar caracteres especiales y convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Elimnar los caracteres que no coincidan con el patrón
    text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', text)


    # Tokenizar el texto
    tokens = nltk.word_tokenize(text)

    # Eliminar stopwords en inglés
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lematizar los tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in filtered_tokens]

    # Unir los tokens procesados en un texto procesado
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

# Obtener la columna 'text' del DataFrame 'train' y 'test' donde 'language' es 'en'

texts_train = train.loc[train['language'] == 'en', 'text']
texts_test = test.loc[test['language'] == 'en', 'text']

# Aplicar el preprocesamiento a cada texto en la columna 'text'
preprocessed_texts_train = [preprocess_text(text) for text in texts_train]
preprocessed_texts_test = [preprocess_text(text) for text in texts_test]

# Crear un nuevo DataFrame con las columnas 'text' y 'task1' para train y test
en_df_train = pd.DataFrame({'text': preprocessed_texts_train, 'task2': train.loc[train['language'] == 'en', 'task2']})
en_df_test = pd.DataFrame({'text': preprocessed_texts_test, 'task2': test.loc[test['language'] == 'en', 'task2']})


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Pre-procesamiento para el idioma Español.
import re
import spacy
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

# Descargar el modelo de SpaCy para el idioma español
spacy.cli.download("es_core_news_sm")

# Cargar el modelo de SpaCy para el idioma español
nlp = spacy.load("es_core_news_sm")

# Resto del código...

# Función para tokenizar y lematizar el texto utilizando SpaCy
def preprocess_text(text):
    # Eliminar usuarios identificados por @
    text = re.sub(r'@\S+', '', text)

    # Elimina URL's y links
    text = re.sub(r'https?:\/\/\S+', '', text)

    text = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', text)

    # Elimina referencias link o video

    text = re.sub(r'{link}', '', text)
    text = re.sub(r"\[video\]", '', text)

    # Elimina entidades HTML
    text = re.sub(r'&[a-z]+;', '', text)

    # Eliminar caracteres especiales y convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

    # Elimnar los caracteres que no coincidan con el patrón
    text = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', text)

    # Tokenizar el texto utilizando SpaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]

    # Eliminar stopwords en español
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Unir los tokens procesados en un texto procesado
    processed_text = ' '.join(filtered_tokens)

    return processed_text



# Obtener la columna 'text' del DataFrame 'train' donde 'language' es 'es'
texts_train = train[train['language'] == 'es']['text']
texts_test = test[test['language'] == 'es']['text']

# Aplicar el preprocesamiento a cada texto en la columna 'text'
preprocessed_texts_train = [preprocess_text(text) for text in texts_train]
preprocessed_texts_test = [preprocess_text(text) for text in texts_test]

# Crear un nuevo DataFrame con la columna 'pre
# Crear un nuevo DataFrame con las columnas 'text' y 'task1' para train y test
es_df_train = pd.DataFrame({'text': preprocessed_texts_train, 'task2': train.loc[train['language'] == 'es', 'task2']})
es_df_test = pd.DataFrame({'text': preprocessed_texts_test, 'task2': test.loc[test['language'] == 'es', 'task2']})

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [6]:
# Combinación de los dataframes train y test de cada idioma
combined_df_train = pd.concat([en_df_train, es_df_train])
combined_df_train = combined_df_train.sample(frac=1).reset_index(drop=True)

combined_df_test = pd.concat([en_df_test, es_df_test])
combined_df_test = combined_df_test.sample(frac=1).reset_index(drop=True)

In [38]:
#train_data = combined_df_train['text']
#test_data = combined_df_test['text']
Y_train = combined_df_train['task2']
Y_test = combined_df_test['task2']

In [39]:
# Preprocesamiento de los datos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_df_train['text'])  # train_data es una lista de textos de entrenamiento

X_train = tokenizer.texts_to_sequences(combined_df_train['text'])
X_test = tokenizer.texts_to_sequences(combined_df_test['text'])


In [40]:

vocab_size = len(tokenizer.word_index) + 1

In [41]:
print(X_train[:5])  # Imprimir los primeros 5 elementos de X_train
print(X_test[:5])  # Imprimir los primeros 5 elementos de X_test

[[2047, 8, 1, 652, 325, 8031, 138, 1507, 2708, 516, 1000, 274, 1186, 38, 890, 8032, 18, 3996, 109, 5254, 8033, 295, 947, 5255, 470, 2337, 728, 1826], [1, 82, 1286, 2048, 1508, 5256, 2709, 387, 8, 17, 24, 51, 5257, 9, 186, 8034, 8035, 8036], [1287, 8037, 1288, 3250, 8038], [3, 3997, 412, 144, 3, 3997, 1117, 8039, 10, 27, 3251, 218, 678, 3252, 8040, 1827, 8041, 73, 8042, 4, 445, 948, 8043, 8044, 8045, 8046, 630, 2338], [1374, 838, 1118, 102, 173, 54]]
[[43, 86, 3, 1090, 260, 28, 981, 1050, 453, 236, 260, 233], [69, 17916, 71, 350, 6381, 2178, 6546, 794, 288], [1336, 833, 4942, 2203, 601, 7, 109, 14, 922, 4379, 4942, 1, 2, 67, 220, 193, 2, 2203, 601], [58, 941, 181, 370, 542, 1006, 1133, 877, 6432, 288, 181, 3, 501, 45, 941], [674, 211, 5423, 3, 243, 10, 5083, 700, 182, 33, 1269, 11541, 53, 21]]


In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len = 255  # longitud máxima de las secuencias
X_train_padded = pad_sequences(X_train, maxlen=max_len)
X_test_padded = pad_sequences(X_test, maxlen=max_len)


In [43]:
import numpy as np

num_classes = len(np.unique(Y_train_encoded))
print(num_classes)

6


In [44]:
#from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#Y_train_encoded = label_encoder.fit_transform(combined_df_train['task2'])
#Y_test_encoded = label_encoder.transform(combined_df_test['task2'])

from sklearn.preprocessing import LabelEncoder
num_classes = 6
# Crear una instancia de LabelEncoder
label_encoder = LabelEncoder()

# Codificar las etiquetas de texto en valores numéricos
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Aplicar one-hot encoding a las etiquetas codificadas
Y_train_onehot = to_categorical(Y_train_encoded, num_classes=num_classes)
Y_test_onehot = to_categorical(Y_test_encoded, num_classes=num_classes)



In [45]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

num_classes = 6  # Número de clases en tu conjunto de datos

# Construcción del modelo LSTM
model = Sequential()
model.add(Embedding(vocab_size, 255, input_length=max_len))
model.add(LSTM(255))
model.add(Dense(num_classes, activation='softmax'))  # num_classes es el número de clases en tu conjunto de datos

In [50]:
# Compilación del modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
print(X_train)


[[2047, 8, 1, 652, 325, 8031, 138, 1507, 2708, 516, 1000, 274, 1186, 38, 890, 8032, 18, 3996, 109, 5254, 8033, 295, 947, 5255, 470, 2337, 728, 1826], [1, 82, 1286, 2048, 1508, 5256, 2709, 387, 8, 17, 24, 51, 5257, 9, 186, 8034, 8035, 8036], [1287, 8037, 1288, 3250, 8038], [3, 3997, 412, 144, 3, 3997, 1117, 8039, 10, 27, 3251, 218, 678, 3252, 8040, 1827, 8041, 73, 8042, 4, 445, 948, 8043, 8044, 8045, 8046, 630, 2338], [1374, 838, 1118, 102, 173, 54], [41, 6, 204, 25, 3998], [436, 12, 1289, 114, 5258, 19, 3253, 12, 2710, 631, 5259, 8047, 490, 10, 30, 12, 949, 232, 10, 3999, 117, 3, 949], [56, 563, 413, 10, 30, 3, 3], [106, 10, 163, 8048, 5260, 218, 47, 529, 107, 208, 437, 4000, 159, 93, 8049, 5261, 8050, 564, 13, 33], [2049, 1287, 14, 109, 2339, 5262, 2050, 2051, 2340, 344, 4001, 1047], [19, 4002, 1375, 10, 414, 20, 1828, 52, 8051, 10, 950, 1187, 704, 705, 1376, 3, 3, 3254, 8052, 1119, 3255], [3256, 218, 2341, 49, 1509, 8053, 8054, 8055, 729, 3256, 5263, 50, 211, 92, 218, 164, 233], [57,

In [48]:
print(Y_train_onehot)

[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [51]:
# Entrenamiento del modelo
model.fit(X_train_padded, Y_train_onehot, batch_size=32, epochs=10, validation_data=(X_test_padded, Y_test_onehot))

Epoch 1/10
219/219 [==============================] - 367s 2s/step - loss: 1.3664 - accuracy: 0.5381 - val_loss: 1.2526 - val_accuracy: 0.5703
Epoch 2/10
219/219 [==============================] - 365s 2s/step - loss: 0.7087 - accuracy: 0.7558 - val_loss: 1.3819 - val_accuracy: 0.5524
Epoch 3/10
219/219 [==============================] - 361s 2s/step - loss: 0.2366 - accuracy: 0.9235 - val_loss: 1.6951 - val_accuracy: 0.5343
Epoch 4/10
219/219 [==============================] - 366s 2s/step - loss: 0.0841 - accuracy: 0.9746 - val_loss: 2.1345 - val_accuracy: 0.5213
Epoch 5/10
190/219 [=========================>....] - ETA: 37s - loss: 0.0455 - accuracy: 0.9865

KeyboardInterrupt: ignored

In [ ]:
Y_pred = model.predict(X_test)


In [ ]:
# Evaluación del modelo
loss, accuracy = model.evaluate(X_test, Y_test_onehot)
print("Loss:", loss)
print("Accuracy:", accuracy)

En una primera prueba el modelo entra en overfitting claramente.